In [1]:
import json
import cv2
import numpy as np
from pycocotools.coco import COCO
import os
import mediapipe as mp

In [2]:
#데이터셋 경로
data_dir='./Cocodataset'
ann_file = f'{data_dir}/annotations/person_keypoints_train2017.json'

In [3]:
#데이터 로드
coco = COCO(ann_file)

loading annotations into memory...
Done (t=7.80s)
creating index...
index created!


In [4]:
#이미지 어노테이션 불러오기 (사람 카테고리 id)
img_ids = coco.getImgIds(catIds=[1])
images = coco.loadImgs(img_ids)

In [5]:
#데이터셋 초기화
X=[]
y=[]

In [6]:

# turtle_neck 폴더 경로
turtle_neck_folder = f"{data_dir}/turtle_neck"
image_files = os.listdir(turtle_neck_folder)

# Mediapipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

X = []
y = []

# 거북목 라벨링 함수
def calculate_neck_angle(nose, left_shoulder, right_shoulder):
    shoulder_center_x = (left_shoulder[0] + right_shoulder[0]) / 2
    shoulder_center_y = (left_shoulder[1] + right_shoulder[1]) / 2
    delta_x = nose[0] - shoulder_center_x
    delta_y = shoulder_center_y - nose[1]
    angle_radians = np.arctan2(delta_y, delta_x)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

# 데이터셋 처리: turtle_neck 폴더의 이미지 순회
for img_file in image_files:
    img_path = os.path.join(turtle_neck_folder, img_file)
    img = cv2.imread(img_path)
    
    # 이미지가 제대로 로드되었는지 확인
    if img is not None:
        # Mediapipe로 이미지에서 keypoints 추출
        image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        if results.pose_landmarks:
            # 필요한 keypoints 추출 (코, 왼쪽 어깨, 오른쪽 어깨)
            nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
            left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
            right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            
            # keypoints의 (x, y) 좌표 추출
            nose_coords = (nose.x, nose.y)
            left_shoulder_coords = (left_shoulder.x, left_shoulder.y)
            right_shoulder_coords = (right_shoulder.x, right_shoulder.y)
            
            # 목 각도 계산
            angle = calculate_neck_angle(nose_coords, left_shoulder_coords, right_shoulder_coords)
            
            # 임계값 기준 거북목 여부 결정
            if angle > 45:
                y.append(1)  # 거북목
            else:
                y.append(0)  # 정상 자세
            
            # 이미지 크기 조정 및 데이터셋 추가
            img_resized = cv2.resize(img, (64, 64))  # 이미지 크기 조정
            X.append(img_resized)
        else:
            print(f"Keypoints를 추출할 수 없습니다: {img_path}")
    else:
        print(f"이미지를 불러올 수 없습니다: {img_path}")

# numpy 배열로 저장
X = np.array(X)
y = np.array(y)
np.save('X.npy', X)
np.save('y.npy', y)

print("라벨링 완료 및 데이터 저장")


C:\Users\goode\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4076.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4112.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4113.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4114.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4115.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4116.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4117.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4118.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4119.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4120.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4121.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4122.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4123.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4124.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4125.JPG
Keypoints를 추출할 수 없습니다: ./Cocodataset/turtle_neck\IMG_4126.JPG
Keypoint

In [7]:
np.save('X.npy', np.array(X))
np.save('y.npy', np.array(y))

print("라벨링 완료 및 데이터 저장")

라벨링 완료 및 데이터 저장
